### Preliminary setup

In [1]:
# For data transformation
import pandas as pd            
# For statistical analysis
import numpy as np
import statistics as stats
# For ASN lookup
import pyasn
asndb = pyasn.pyasn('ipasn_20140513.dat')

In [2]:
# input csv
input_file = "./cesnet_dataset.csv"
new_df = pd.read_csv(input_file, sep=",", low_memory=False, header= 0)
#df = pd.read_csv(input_file, dtype={"DURATION": float, "SRC_IP": str, "DST_IP": str, "SRC_PORT": int, "DST_PORT": int}, sep=",", low_memory=False, header= 0)

# delete protocol column
# df = df.drop(columns=["PROTOCOL"], axis=1)
# df["BYTES"] = pd.to_numeric(df["BYTES"], errors='coerce').fillna(0)

# Read the existing CSV file
output_file = "./benign_flow/benign.csv"

In [16]:
# set source IP, web service category, and label
ipsrc = "10.10.3.10"
label = "0"
quic_ver = "1"

# define the list for each feature
df_dst_port = []
df_dst_asn = []
df_dur = []
df_ratio = []
df_flow_pkt = []
df_flow_bytes = []
df_tot_pkt = []
df_tot_bytes = []
df_max_bytes = []
df_min_bytes = []
df_ave_bytes = []
df_std_bytes = []
df_var_bytes = []
df_fwd_pkt = []
df_fwd_bytes = []
df_max_fwd_bytes = []
df_min_fwd_bytes = []
df_ave_fwd_bytes = []
df_std_fwd_bytes = []
df_var_fwd_bytes = []
df_rev_pkt = []
df_rev_bytes = []
df_max_rev_bytes = []
df_min_rev_bytes = []
df_ave_rev_bytes = []
df_std_rev_bytes = []
df_var_rev_bytes = []
df_max_iat = []
df_min_iat = []
df_ave_iat = []
df_std_iat = []
df_var_iat = []
df_fwd_dur = []
df_max_fwd_iat = []
df_min_fwd_iat = []
df_ave_fwd_iat = []
df_std_fwd_iat = []
df_var_fwd_iat = []
df_rev_dur = []
df_max_rev_iat = []
df_min_rev_iat = []
df_ave_rev_iat = []
df_std_rev_iat = []
df_var_rev_iat = []
df_label = []

# initialize the interpacket variables
arr_port = []
arr_asn = []
arr_ver = []
def initialize_variables():
    global arr_fwd_bytes, arr_rev_bytes, arr_fwd_iat, arr_rev_iat, arr_port, arr_asn, arr_ver
    global fwd_pkt, fwd_bytes, rev_pkt, rev_bytes, init_dur, dur, fwd_dur, rev_dur, ctr
    arr_fwd_bytes = []
    arr_rev_bytes = []
    arr_fwd_iat = []
    arr_rev_iat = []
    fwd_pkt = 0
    fwd_bytes = 0.0
    rev_pkt = 0
    rev_bytes = 0.0
    init_dur = 0.0
    dur = 0.0
    fwd_dur = 0.0
    rev_dur = 0.0
    ctr = 0

def get_asn(ip):
    try:
        return asndb.lookup(ip)[0]
    except:
        false_ip = ip.split(",")[0]
        return asndb.lookup(false_ip)[0]

initialize_variables()
for index, row in df.iterrows():
    value = row['SRC_IP']
    if ctr == 29 or index == len(df) - 1 or (index > 0 and abs(df.at[index, 'DURATION'] - df.at[index - 1, 'DURATION']) > 1.0):
        if value.startswith(ipsrc):
            fwd_dur += (row['DURATION'] - init_dur)
            arr_fwd_iat.append(row['DURATION'] - init_dur)
            arr_port.append(row['DST_PORT'])
            arr_asn.append(get_asn(row['DST_IP']))
            arr_ver.append(row['QUIC_VERSION'])
            fwd_pkt += 1
            fwd_bytes += row['BYTES']
            arr_fwd_bytes.append(row['BYTES'])
        else:
            rev_dur += (row['DURATION'] - init_dur)
            arr_rev_iat.append(row['DURATION'] - init_dur)
            arr_port.append(row['SRC_PORT'])
            arr_asn.append(get_asn(row['SRC_IP']))
            arr_ver.append(row['QUIC_VERSION'])
            rev_pkt += 1
            rev_bytes += row['BYTES']
            arr_rev_bytes.append(row['BYTES'])

        dst_port = stats.mode(arr_port)
        dst_asn = stats.mode([x for x in arr_asn if isinstance(x, int)])
        new_quic_ver = stats.mode(arr_ver)

        if isinstance(new_quic_ver, tuple):
            quic_ver = max([int(h) for h in new_quic_ver])
        elif isinstance(new_quic_ver, int):
            quic_ver = [int(new_quic_ver)]
        
        dur += (row['DURATION'] - init_dur)
        ratio = 1 if rev_pkt > fwd_pkt else 0
        tot_pkt = fwd_pkt + rev_pkt
        tot_bytes = fwd_bytes + rev_bytes
        flow_pkt = tot_pkt / dur
        flow_bytes = tot_bytes / dur
        
        if len(arr_fwd_bytes) > 0 and len(arr_rev_bytes) > 0:
            combined_bytes = np.array(arr_fwd_bytes + arr_rev_bytes)
            max_bytes = np.max(combined_bytes)
            min_bytes = np.min(combined_bytes)
            ave_bytes = np.mean(combined_bytes)
            std_bytes = np.std(combined_bytes)
            var_bytes = np.var(combined_bytes)

        if len(arr_fwd_bytes) > 0:
            max_fwd_bytes = np.max(arr_fwd_bytes)
            min_fwd_bytes = np.min(arr_fwd_bytes)
            ave_fwd_bytes = np.mean(arr_fwd_bytes)
            std_fwd_bytes = np.std(arr_fwd_bytes)
            var_fwd_bytes = np.var(arr_fwd_bytes)

        if len(arr_rev_bytes) > 0:
            max_rev_bytes = np.max(arr_rev_bytes)
            min_rev_bytes = np.min(arr_rev_bytes)
            ave_rev_bytes = np.mean(arr_rev_bytes)
            std_rev_bytes = np.std(arr_rev_bytes)
            var_rev_bytes = np.var(arr_rev_bytes)

        if len(arr_fwd_iat) > 0 and len(arr_rev_iat) > 0:
            combined_iat = np.array(arr_fwd_iat + arr_rev_iat)
            max_iat = np.max(combined_iat)
            min_iat = np.min(combined_iat)
            ave_iat = np.mean(combined_iat)
            std_iat = np.std(combined_iat)
            var_iat = np.var(combined_iat)

        if len(arr_fwd_iat) > 0:
            max_fwd_iat = np.max(arr_fwd_iat)
            min_fwd_iat = np.min(arr_fwd_iat)
            ave_fwd_iat = np.mean(arr_fwd_iat)
            std_fwd_iat = np.std(arr_fwd_iat)
            var_fwd_iat = np.var(arr_fwd_iat)

        if len(arr_rev_iat) > 0:
            max_rev_iat = np.max(arr_rev_iat)
            min_rev_iat = np.min(arr_rev_iat)
            ave_rev_iat = np.mean(arr_rev_iat)
            std_rev_iat = np.std(arr_rev_iat)
            var_rev_iat = np.var(arr_rev_iat)

        df_dst_port.append(dst_port)
        df_dst_asn.append(dst_asn)
        df_dur.append(dur)
        df_ratio.append(ratio)
        df_flow_pkt.append(flow_pkt)
        df_flow_bytes.append(flow_bytes)
        df_tot_pkt.append(tot_pkt)
        df_tot_bytes.append(tot_bytes)
        df_max_bytes.append(max_bytes)
        df_min_bytes.append(min_bytes)
        df_ave_bytes.append(ave_bytes)
        df_std_bytes.append(std_bytes)
        df_var_bytes.append(var_bytes)
        df_fwd_pkt.append(fwd_pkt)
        df_fwd_bytes.append(fwd_bytes)
        df_max_fwd_bytes.append(max_fwd_bytes)
        df_min_fwd_bytes.append(min_fwd_bytes)
        df_ave_fwd_bytes.append(ave_fwd_bytes)
        df_std_fwd_bytes.append(std_fwd_bytes)
        df_var_fwd_bytes.append(var_fwd_bytes)
        df_rev_pkt.append(rev_pkt)
        df_rev_bytes.append(rev_bytes)
        df_max_rev_bytes.append(max_rev_bytes)
        df_min_rev_bytes.append(min_rev_bytes)
        df_ave_rev_bytes.append(ave_rev_bytes)
        df_std_rev_bytes.append(std_rev_bytes)
        df_var_rev_bytes.append(var_rev_bytes)
        df_max_iat.append(max_iat)
        df_min_iat.append(min_iat)
        df_ave_iat.append(ave_iat)
        df_std_iat.append(std_iat)
        df_var_iat.append(var_iat)
        df_fwd_dur.append(fwd_dur)
        df_max_fwd_iat.append(max_fwd_iat)
        df_min_fwd_iat.append(min_fwd_iat)
        df_ave_fwd_iat.append(ave_fwd_iat)
        df_std_fwd_iat.append(std_fwd_iat)
        df_var_fwd_iat.append(var_fwd_iat)
        df_rev_dur.append(rev_dur)
        df_max_rev_iat.append(max_rev_iat)
        df_min_rev_iat.append(min_rev_iat)
        df_ave_rev_iat.append(ave_rev_iat)
        df_std_rev_iat.append(std_rev_iat)
        df_var_rev_iat.append(var_rev_iat)
        df_label.append(label) 

        initialize_variables()
    elif value.startswith(ipsrc) and ctr < 29:
        if ctr == 0:
            init_dur = row['DURATION']
        else:
            fwd_dur += (row['DURATION'] - init_dur)
            arr_fwd_iat.append(row['DURATION'] - init_dur)
        ctr += 1
        arr_port.append(row['DST_PORT'])
        arr_asn.append(get_asn(row['DST_IP']))
        arr_ver.append(row['QUIC_VERSION'])
        fwd_pkt += 1
        fwd_bytes += row['BYTES']
        arr_fwd_bytes.append(row['BYTES'])
    elif value.startswith(ipsrc) == False and ctr < 29:
        if ctr == 0:
            init_dur = row['DURATION']
        else:
            rev_dur += (row['DURATION'] - init_dur)
            arr_rev_iat.append(row['DURATION'] - init_dur)
        ctr += 1
        arr_port.append(row['SRC_PORT'])
        arr_asn.append(get_asn(row['SRC_IP']))
        arr_ver.append(row['QUIC_VERSION'])
        rev_pkt += 1
        rev_bytes += row['BYTES']
        arr_rev_bytes.append(row['BYTES'])
    
    # track progress in percent with respect to total rows
    if index % 1000 == 0:
        print(f"{index / len(df) * 100:.2f}%")



0.00%
6.96%
13.91%
20.87%
27.83%
34.79%
41.74%
48.70%
55.66%
62.62%
69.57%
76.53%
83.49%
90.45%
97.40%


In [4]:
# for item in flow:
#     print(item)

print(len(new_df))

7228974


In [18]:
# Create a list to store all flow records
flow_rows = []

# Loop through all indices
for i in range(len(df_dst_port)):
    # Create dictionary for each row
    flow_row = {
        'dst_port': df_dst_port[i],
        'dst_asn': df_dst_asn[i],
        'quic_ver': quic_ver,
        'dur': df_dur[i],
        'ratio': df_ratio[i],
        'flow_pkt_rate': df_flow_pkt[i],
        'flow_byte_rate': df_flow_bytes[i],
        'total_pkts': df_tot_pkt[i],
        'total_bytes': df_tot_bytes[i],
        'max_bytes': df_max_bytes[i],
        'min_bytes': df_min_bytes[i],
        'ave_bytes': df_ave_bytes[i],
        'std_bytes': df_std_bytes[i],
        'var_bytes': df_var_bytes[i],
        'fwd_pkts': df_fwd_pkt[i],
        'fwd_bytes': df_fwd_bytes[i],
        'max_fwd_bytes': df_max_fwd_bytes[i],
        'min_fwd_bytes': df_min_fwd_bytes[i],
        'ave_fwd_bytes': df_ave_fwd_bytes[i],
        'std_fwd_bytes': df_std_fwd_bytes[i],
        'var_fwd_bytes': df_var_fwd_bytes[i],
        'rev_pkts': df_rev_pkt[i],
        'rev_bytes': df_rev_bytes[i],
        'max_rev_bytes': df_max_rev_bytes[i],
        'min_rev_bytes': df_min_rev_bytes[i],
        'ave_rev_bytes': df_ave_rev_bytes[i],
        'std_rev_bytes': df_std_rev_bytes[i],
        'var_rev_bytes': df_var_rev_bytes[i],
        'max_iat': df_max_iat[i],
        'min_iat': df_min_iat[i],
        'ave_iat': df_ave_iat[i],
        'std_iat': df_std_iat[i],
        'var_iat': df_var_iat[i],
        'fwd_dur': df_fwd_dur[i],
        'max_fwd_iat': df_max_fwd_iat[i],
        'min_fwd_iat': df_min_fwd_iat[i],
        'ave_fwd_iat': df_ave_fwd_iat[i],
        'std_fwd_iat': df_std_fwd_iat[i],
        'var_fwd_iat': df_var_fwd_iat[i],
        'rev_dur': df_rev_dur[i],
        'max_rev_iat': df_max_rev_iat[i],
        'min_rev_iat': df_min_rev_iat[i],
        'ave_rev_iat': df_ave_rev_iat[i],
        'std_rev_iat': df_std_rev_iat[i],
        'var_rev_iat': df_var_rev_iat[i],
        'label': df_label[i]
    }
    flow_rows.append(flow_row)

# Create DataFrame from the list of dictionaries
new_flow_df = pd.DataFrame(flow_rows)

In [19]:
file_path = "./benign_flow/benign.csv"

# Write the updated DataFrame back to the CSV file
new_flow_df.to_csv(file_path, index=False)

### Helper Functions

In [ ]:
# Cell to inspect DataFrame contents and diagnose issues

# Check basic DataFrame info
print(f"DataFrame shape: {df.shape}")
print(f"DataFrame columns: {df.columns.tolist()}")
print("\nSample of first 5 rows:")
print(df.head().to_string())

# Check for commas in IP fields
src_ip_with_commas = df[df['SRC_IP'].str.contains(',', na=False)]
dst_ip_with_commas = df[df['DST_IP'].str.contains(',', na=False)]

print(f"\nRows with commas in SRC_IP: {len(src_ip_with_commas)}")
print(f"Rows with commas in DST_IP: {len(dst_ip_with_commas)}")

if len(src_ip_with_commas) > 0:
    print("\nSample of rows with commas in SRC_IP:")
    print(src_ip_with_commas.head().to_string())
    
    # Count unique values with commas
    unique_problematic_src_ips = src_ip_with_commas['SRC_IP'].unique()
    print(f"\nUnique problematic SRC_IP values ({len(unique_problematic_src_ips)}):")
    for ip in unique_problematic_src_ips[:10]:  # Show first 10
        print(f"  - {ip}")
    if len(unique_problematic_src_ips) > 10:
        print(f"  ... and {len(unique_problematic_src_ips) - 10} more")

# Check specific comparison that's failing
ipsrc = "10.10.3.10"
problem_rows = df[df['SRC_IP'].str.startswith(ipsrc) & (df['SRC_IP'] != ipsrc)]
print(f"\nRows where SRC_IP starts with '{ipsrc}' but isn't exactly '{ipsrc}': {len(problem_rows)}")
if len(problem_rows) > 0:
    print("\nSample of these problematic rows:")
    print(problem_rows.head().to_string())

# Compare string values and lengths
if len(problem_rows) > 0:
    example = problem_rows['SRC_IP'].iloc[0]
    print(f"\nDetailed comparison:")
    print(f"Expected: '{ipsrc}' (length: {len(ipsrc)})")
    print(f"Actual  : '{example}' (length: {len(example)})")
    print(f"Equal?  : {ipsrc == example}")
    
    # Check character by character
    print("\nCharacter by character comparison:")
    for i in range(max(len(ipsrc), len(example))):
        if i < len(ipsrc) and i < len(example):
            match = ipsrc[i] == example[i]
            print(f"Position {i}: '{ipsrc[i]}' vs '{example[i]}' - Match: {match}")
        elif i < len(ipsrc):
            print(f"Position {i}: '{ipsrc[i]}' vs (no character) - No match")
        else:
            print(f"Position {i}: (no character) vs '{example[i]}' - No match")

In [ ]:
# Analyze dataset

import pandas as pd
import os
import glob
import argparse

def count_total_packets(input_paths, verbose=True):
    """
    Count the total number of packets across all flows in the given CSV file(s)
    
    Args:
        input_paths (str or list): Path(s) to CSV file(s) or directory containing CSVs
        verbose (bool): Whether to print detailed information
    
    Returns:
        dict: Dictionary with packet count statistics
    """
    # Handle file paths
    if isinstance(input_paths, str):
        # If it's a directory, get all CSV files
        if os.path.isdir(input_paths):
            file_paths = glob.glob(os.path.join(input_paths, "*.csv"))
        else:
            file_paths = [input_paths]
    else:
        file_paths = input_paths
    
    total_flows = 0
    total_packets = 0
    total_fwd_packets = 0
    total_rev_packets = 0
    total_duration = 0
    total_bytes = 0
    
    # Process each file
    for file_path in file_paths:
        if verbose:
            print(f"Processing {file_path}...")
        
        try:
            # Check if this is a flow feature file (with our specific columns)
            # or raw packet data
            df = pd.read_csv(file_path, nrows=1)
            
            if "total_pkts" in df.columns or "PACKETS" in df.columns:
                # This is a flow feature file
                df = pd.read_csv(file_path)
                
                # Get packet counts based on available columns
                if "total_pkts" in df.columns:
                    flows_in_file = len(df)
                    packets_in_file = df["total_pkts"].sum()
                    fwd_packets_in_file = df["fwd_pkts"].sum() if "fwd_pkts" in df.columns else 0
                    rev_packets_in_file = df["rev_pkts"].sum() if "rev_pkts" in df.columns else 0
                    duration_in_file = df["dur"].sum() if "dur" in df.columns else 0
                    bytes_in_file = df["total_bytes"].sum() if "total_bytes" in df.columns else 0
                
                elif "PACKETS" in df.columns:
                    flows_in_file = len(df)
                    packets_in_file = df["PACKETS"].sum() + df["PACKETS_REV"].sum() if "PACKETS_REV" in df.columns else df["PACKETS"].sum()
                    fwd_packets_in_file = df["PACKETS"].sum()
                    rev_packets_in_file = df["PACKETS_REV"].sum() if "PACKETS_REV" in df.columns else 0
                    duration_in_file = df["DURATION"].sum() if "DURATION" in df.columns else 0
                    
                    # Handle bytes based on available columns
                    if "BYTES" in df.columns and "BYTES_REV" in df.columns:
                        bytes_in_file = df["BYTES"].sum() + df["BYTES_REV"].sum()
                    elif "BYTES" in df.columns:
                        bytes_in_file = df["BYTES"].sum()
                    else:
                        bytes_in_file = 0
                
                if verbose:
                    print(f"  - {flows_in_file} flows found")
                    print(f"  - {packets_in_file} total packets")
                    print(f"  - {fwd_packets_in_file} forward packets")
                    print(f"  - {rev_packets_in_file} reverse packets")
                    print(f"  - {duration_in_file:.2f} seconds total duration")
                    print(f"  - {bytes_in_file:,} bytes transferred")
                
                total_flows += flows_in_file
                total_packets += packets_in_file
                total_fwd_packets += fwd_packets_in_file
                total_rev_packets += rev_packets_in_file
                total_duration += duration_in_file
                total_bytes += bytes_in_file
            
            else:
                # This might be a raw packet file
                if verbose:
                    print("  This appears to be raw packet data, counting rows...")
                
                # Count rows (each row is a packet)
                row_count = sum(1 for _ in open(file_path)) - 1  # subtract header
                
                if verbose:
                    print(f"  - {row_count} packets found")
                
                total_packets += row_count
        
        except Exception as e:
            print(f"Error processing {file_path}: {e}")
    
    # Print summary
    if verbose:
        print("\n===== SUMMARY =====")
        print(f"Total flows: {total_flows:,}")
        print(f"Total packets: {total_packets:,}")
        print(f"Forward packets: {total_fwd_packets:,}")
        print(f"Reverse packets: {total_rev_packets:,}")
        print(f"Total duration: {total_duration:.2f} seconds ({total_duration/3600:.2f} hours)")
        print(f"Total bytes: {total_bytes:,} ({total_bytes/(1024*1024):.2f} MB)")
        
        if total_flows > 0:
            print(f"Average packets per flow: {total_packets/total_flows:.2f}")
            print(f"Average duration per flow: {total_duration/total_flows:.2f} seconds")
            print(f"Average bytes per flow: {total_bytes/total_flows:,.2f}")
        else:
            print("No flows found")
            
        if total_duration > 0:
            print(f"Average packets per second: {total_packets/total_duration:.2f}")
            print(f"Average bandwidth: {(total_bytes*8)/(total_duration*1000):.2f} Kbps")
        
        print("===================")
    
    # Return as dictionary
    return {
        "total_flows": total_flows,
        "total_packets": total_packets,
        "forward_packets": total_fwd_packets,
        "reverse_packets": total_rev_packets,
        "total_duration": total_duration,
        "total_bytes": total_bytes,
        "avg_packets_per_flow": total_packets/total_flows if total_flows > 0 else 0,
        "avg_duration_per_flow": total_duration/total_flows if total_flows > 0 else 0,
        "avg_bytes_per_flow": total_bytes/total_flows if total_flows > 0 else 0,
        "avg_packets_per_second": total_packets/total_duration if total_duration > 0 else 0,
        "avg_bandwidth_kbps": (total_bytes*8)/(total_duration*1000) if total_duration > 0 else 0
    }

if __name__ == "__main__":
    # Parse command line arguments
    parser = argparse.ArgumentParser(description='Count total packets and duration in flow CSV files')
    parser.add_argument('--input', '-i', required=True, help='Input CSV file path or directory containing CSVs')
    parser.add_argument('--quiet', '-q', action='store_true', help='Reduce output verbosity')
    parser.add_argument('--output', '-o', help='Save results to CSV file')
    
    args = parser.parse_args()
    
    # Count packets and duration
    results = count_total_packets(args.input, verbose=not args.quiet)
    
    # Save to CSV if requested
    if args.output:
        # Convert to DataFrame for easy CSV export
        df_results = pd.DataFrame([results])
        df_results.to_csv(args.output, index=False)
        if not args.quiet:
            print(f"Results saved to {args.output}")

In [ ]:
# Merge all CSV files from multiple folders into one master file

import os
import pandas as pd
import glob
from tqdm import tqdm
import time

def merge_csv_files(parent_folder, output_file='master_file.csv'):
    """
    Merge all CSV files from multiple folders into one master file.
    
    Args:
        parent_folder (str): Path to the parent folder containing subfolders with CSV files
        output_file (str): Path where the merged CSV file will be saved
    """
    start_time = time.time()
    
    # Get all CSV files from all subfolders
    all_csv_files = glob.glob(os.path.join(parent_folder, '**', '*.csv'), recursive=True)
    
    if not all_csv_files:
        print(f"No CSV files found in {parent_folder} or its subfolders.")
        return
    
    print(f"Found {len(all_csv_files)} CSV files in {len(set(os.path.dirname(f) for f in all_csv_files))} folders.")
    
    # Read the first file to get the header
    first_df = pd.read_csv(all_csv_files[0])
    header = first_df.columns.tolist()
    
    # Write the header to the output file
    with open(output_file, 'w', encoding='utf-8') as f_output:
        pd.DataFrame(columns=header).to_csv(f_output, index=False)
    
    # Process each file and append to master file
    print("Merging files...")
    for file_path in tqdm(all_csv_files):
        try:
            # Read each CSV file in chunks to handle large files efficiently
            for chunk in pd.read_csv(file_path, chunksize=100000):
                # Append to the master file without writing the header again
                chunk.to_csv(output_file, mode='a', header=False, index=False)
        except Exception as e:
            print(f"Error processing file {file_path}: {str(e)}")
    
    # Final processing time
    elapsed_time = time.time() - start_time
    print(f"Merge completed in {elapsed_time:.2f} seconds.")
    print(f"Master file saved as: {os.path.abspath(output_file)}")
    
    # Get some basic stats about the merged file
    try:
        file_size_mb = os.path.getsize(output_file) / (1024 * 1024)
        row_count = sum(1 for _ in open(output_file, 'r')) - 1  # Subtract 1 for header
        print(f"Master file size: {file_size_mb:.2f} MB")
        print(f"Total rows: {row_count:,}")
    except Exception as e:
        print(f"Error getting file stats: {str(e)}")

if __name__ == "__main__":
    # Specify the parent folder containing subfolders with CSV files
    parent_folder = input("Enter the path to the parent folder: ").strip()
    
    # Specify output file path
    output_file = input("Enter the path for the master file (or press Enter for 'master_file.csv'): ").strip()
    if not output_file:
        output_file = "master_file.csv"
    
    # Run the merge
    merge_csv_files(parent_folder, output_file)